In [1]:
%matplotlib inline 
#%load_ext autoreload 
%reload_ext autoreload
%autoreload 2

from __future__ import (division, 
                        print_function)


In [2]:
import os
import sys
import copy

import h5py
import numpy as np
import scipy
from scipy.interpolate import interp1d


/Users/RAJ/anaconda/envs/illustris_profiles/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from astropy.io import fits
from astropy import units as u
from astropy.stats import sigma_clip
from astropy.table import Table, Column
from astropy.utils.console import ProgressBar
from astropy.convolution import convolve, Box1DKernel


In [4]:
import matplotlib as mpl
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
from matplotlib.patches import Ellipse
from matplotlib.ticker import NullFormatter, MaxNLocator, FormatStrFormatter
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
tickFormat = FormatStrFormatter('$\mathbf{%g}$') 
mpl.rcParams.update({'xtick.color': 'k'})
mpl.rcParams.update({'ytick.color': 'k'})
mpl.rcParams.update({'font.size': 20})

import cosmology
c=cosmology.Cosmo(H0=70.0, omega_m=0.3, omega_l=0.7, flat=1)



## You need to download Song's repository. 

https://github.com/dr-guangtou/kungpao/tree/master/kungpao

In [6]:
import sys
sys.path.append('/Users/RAJ/github/kungpao') #this will vary based on the file location
from kungpao.galsbp import galSBP
from kungpao.display import display_single, random_cmap
import statsmodels.api as sm
from scipy.interpolate import interp1d 
from kungpao import io
from kungpao import utils
from kungpao import detection
from kungpao import imtools


In [7]:
import sep
from palettable.colorbrewer.sequential import Greys_9, OrRd_9, Blues_9, Purples_9, YlGn_9
BLK = Greys_9.mpl_colormap
ORG = OrRd_9.mpl_colormap
BLU = Blues_9.mpl_colormap
GRN = YlGn_9.mpl_colormap

# These locations would be different

In [8]:
x_images = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf/bin.macosx/x_images.e'
TBL = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf_extern/tables/bin.macosx/x_ttools.e'
ISO = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf_extern/stsdas/bin.macosx/x_isophote.e'

In [9]:
kernel3 = np.asarray([[0.092163, 0.221178, 0.296069, 0.221178, 0.092163],
                      [0.221178, 0.530797, 0.710525, 0.530797, 0.221178],
                      [0.296069, 0.710525, 0.951108, 0.710525, 0.296069],
                      [0.221178, 0.530797, 0.710525, 0.530797, 0.221178],
                      [0.092163, 0.221178, 0.296069, 0.221178, 0.092163]])

In [10]:
def get_pixel_scale(file):
    f = h5py.File(file, 'r')
    map_size = f['config'].attrs['map_range_min']
    n_pixels = f['config'].attrs['map_npixel']
    pixel_scale=2 * (map_size/n_pixels)

    return pixel_scale

In [11]:
def get_ngalaxies(file):
    f = h5py.File(file, 'r')
    n_galaxies = len(f['catsh_id'])
    return n_galaxies


## file locations

In [12]:
Illustris_dir_1 = '/Users/RAJ/summer_work_2018/Illustris'
Illustris_quick_hdf5 = os.path.join(Illustris_dir_1, 'galaxies_stellarmaps_orig_11.2.hdf5')
Illustris_high_hdf5 = os.path.join(Illustris_dir_1, 'galaxies_stellarmaps_orig_11.2_highres.hdf5')

# Directories for the low res and high resolution maps

In [13]:
illustris_quick_direc = '/Users/RAJ/summer_work_2018/prof_2/quick'
illustris_high_direc ='/Users/RAJ/summer_work_2018/prof_2/high_res'

In [14]:
#getting the pixel scales for the high resolution and low resolution
pixel_scale_high_res = get_pixel_scale(Illustris_high_hdf5)
pixel_scale_quick = get_pixel_scale(Illustris_quick_hdf5)

In [15]:
def save_to_fits(image, name):
    """
    Save a 2-D array as fits image.
    """
    hdu = fits.PrimaryHDU(image)
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(name, overwrite=True)

    return

In [16]:
def get_mass_maps(sim_file,direc):

    f = h5py.File(sim_file, 'r')
    n_galaxies = len(f['catsh_id'])
    cen_insitu = np.array(f['map_star_rho_insitu_xy'])
    cen_exsitu = np.array(f['map_star_rho_exsitu_xy'])
    map_stars_cen = cen_exsitu + cen_insitu 
    fuzz_insitu = np.array(f['map_star_rho_fuzz_insitu_xy'])
    fuzz_exsitu = np.array(f['map_star_rho_fuzz_exsitu_xy'])
    map_stars_fuzz = fuzz_exsitu + fuzz_insitu
    sats_insitu = np.array(f['map_star_rho_oshs_insitu_xy'])
    sats_exsitu = np.array(f['map_star_rho_oshs_exsitu_xy'])
    map_stars_sats = sats_exsitu + sats_insitu
    map_size = f['config'].attrs['map_range_min']
    n_pixels = f['config'].attrs['map_npixel']
    pixel_scale=2 * (map_size/n_pixels)
    f.close()
    Illustris_maps = []
    
    for gal_n in range(n_galaxies):
        img_cen = map_stars_cen[gal_n] * (pixel_scale ** 2) # Central
        img_ins = cen_insitu[gal_n] * (pixel_scale ** 2)
        img_exs = cen_exsitu[gal_n] * (pixel_scale ** 2)
        img_sat = map_stars_sats[gal_n] * (pixel_scale ** 2)
        img_icl = map_stars_fuzz[gal_n] * (pixel_scale ** 2)
        img_all = (img_cen + img_sat + img_icl)
        
        prefix = 'illustris_%s' % str(gal_n).strip()
        
        
        # Central only 
        cen_fits = os.path.join(direc, prefix + "_cen.fits" )
        _ = io.save_to_fits(img_cen, cen_fits)

        # In-situ stars
        ins_fits = os.path.join(direc, prefix + "_ins.fits")
        _ = io.save_to_fits(img_ins, ins_fits)

        # Ex-situ stars
        exs_fits = os.path.join(direc, prefix + "_exs.fits" )
        _ = io.save_to_fits(img_exs, exs_fits)

        # Everything
        all_fits = os.path.join(direc, prefix + "_all.fits" )
        _ = io.save_to_fits(img_all, all_fits)

        Illustris_maps.append(np.stack([img_cen, img_ins, img_exs, 
                                  img_all, img_sat, img_icl]))
    
    return Illustris_maps


In [17]:
def Illustris_aperture_photometry(img, scale, img_cen_x=None, img_cen_y=None):
    """Aperture photometry information for Illustris high resolution galaxy."""
    bkg = sep.Background(img, bw=10, bh=10, fw=5, fh=5)
    img_sub = img - bkg
    objects = sep.extract(img_sub, 20.0, filter_kernel=kernel3)
    
    img_h, img_w = img_sub.shape
    if img_cen_x is None:
        img_cen_x = img_w / 2.0
    if img_cen_y is None:
        img_cen_y = img_h / 2.0

    # Find the object at the center
    index = np.argmin(np.sqrt((objects['x'] - img_cen_x) ** 2.0 + 
                              (objects['y'] - img_cen_y) ** 2.0))

    # Get the naive ba, theta, xcen, ycen
    ba = objects[index]['b'] / objects[index]['a']
    theta = objects[index]['theta']
    xcen, ycen = objects[index]['x'], objects[index]['y']
    
    # Mass within different apertures
    rad = np.asarray([10.0, 20.0, 30.0, 40.0, 50.0, 100.0, 150.0, 200.0])
    maper = sep.sum_ellipse(img, xcen, ycen, rad / scale, rad /scale * ba, theta, 1.0, 
                            bkgann=None, subpix=11)[0]

    aper_results = {'ba': ba, 'pa': theta,
                    'rad': rad, 'maper': np.log10(maper)}
    
    return aper_results

In [19]:
def quick_aperture_photometry(img_arr,aper_results ,img_cen_x=None, img_cen_y=None):
    apr_rs =[]
    for i in range(len(img_arr)):
        bkg = sep.Background(img_arr[i], bw=10, bh=10, fw=5, fh=5)
        img_sub = img_arr[i] - bkg
        objects = sep.extract(img_sub, 20.0, filter_kernel=kernel3)
        ba = aper_results[i]['ba']
        theta = aper_results[i]['pa']
        rad = aper_results[i]['rad']
        maper = aper_results[i]['maper']

        img_h, img_w = img_sub.shape
        if img_cen_x is None:
            img_cen_x = img_w / 2.0
        if img_cen_y is None:
            img_cen_y = img_h / 2.0
        aper_result = {'x': img_cen_x, 'y': img_cen_y, 'ba': ba, 'pa': theta,
                        'rad': rad, 'maper':(maper)}
        apr_rs.append(aper_result)

    return apr_rs


    

In [ ]:
#making the maps
Illustris_maps_quick = get_mass_maps(Illustris_quick_hdf5, illustris_quick_direc)
Illustris_maps_high = get_mass_maps(Illustris_high_hdf5, illustris_high_direc)

In [ ]:
#doing photometry on the high resolution maps
Illustris_0_cen_part = [Illustris_aperture_photometry(gal[0],pixel_scale_high_res) for gal in Illustris_maps_high]
Illustris_0_ins_part = [Illustris_aperture_photometry(gal[1],pixel_scale_high_res) for gal in Illustris_maps_high]
Illustris_0_exs_part = [Illustris_aperture_photometry(gal[2],pixel_scale_high_res) for gal in Illustris_maps_high]

In [18]:
#now we will do photometry on the lowres; we are using results from the highres maps for the shape and pa
images_cen = [gal[0] for gal in Illustris_maps_quick]
images_ins = [gal[1] for gal in Illustris_maps_quick]
images_exs = [gal[2] for gal in Illustris_maps_quick]

Illustris_0_cen_aper = quick_aperture_photometry(images_cen,Illustris_0_cen_part)
Illustris_0_ins_aper = quick_aperture_photometry(images_ins,Illustris_0_ins_part)
Illustris_0_exs_aper = quick_aperture_photometry(images_exs,Illustris_0_exs_part)


In [ ]:
def get_fits_name(idx, img_type='cen', Illustris_dir):
    """Return the name of the FITS file."""
    prefix = 'illustris_%s' % str(idx).strip()
    suffix = '_%s.fits' % (img_type)
    
    return os.path.join(Illustris_dir, prefix + suffix)

def Illustris_get_1d_prof(fits, aper_result, isophote=ISO, xttools=TBL,
                    pixel_scale=pixel_scale_quick):
    """Get Step 2 and Step 3 1-D profile."""
    xcen, ycen = aper_result['x'], aper_result['y']
    ba = aper_result['ba'] 
    pa = utils.normalize_angle(aper_result['pa'] * 180.0 / np.pi + 90.0,
                               lower=-90, upper=90, b=True)
    

    # Step 3 to get mass density profiles
    ell_3, bin_3 = galSBP.galSBP(fits, galX=xcen, galY=ycen, 
                                 maxSma=130, iniSma=8.0, 
                                 verbose=False, savePng=False, 
                                 saveOut=True, expTime=1.0, 
                                 pix=pixel_scale, zpPhoto=0.0,
                                 galQ=ba, galPA=pa, 
                                 stage=3, minSma=0.0, 
                                 ellipStep=0.08,
                                 isophote=isophote, 
                                 xttools=xttools, 
                                 uppClip=2.5, lowClip=3.0, 
                                 maxTry=2, nClip=2, intMode='mean', 
                                 updateIntens=False)
    
    return  ell_3,bin_3
    
    
def Illustris_get_1d_force(fits, binary, aper_result, isophote=ISO, xttools=TBL,
                     pixel_scale=pixel_scale_quick):
    """Get Step 4 force photometry 1-D profile."""
    xcen, ycen = aper_result['x'], aper_result['y']
    ba = aper_result['ba'] 
    pa = utils.normalize_angle(aper_result['pa'] * 180.0 / np.pi + 90.0,
                               lower=-90, upper=90, b=True)
    
    # Step 2 to get ellipticity and position angle profiles
    ell_4, bin_4 = galSBP.galSBP(fits, inEllip=binary, 
                                 galX=xcen, galY=ycen, 
                                 verbose=False, savePng=False, 
                                 saveOut=True, expTime=1.0, 
                                 pix=pixel_scale, zpPhoto=0.0,
                                 stage=4, isophote=isophote, xttools=xttools, 
                                 uppClip=2.5, lowClip=3.0, 
                                 maxTry=2, nClip=2, intMode='mean', 
                                 updateIntens=False)
    
    return ell_4, bin_4


def Illustris_get_1d_all(idx, aper_cen, aper_ins, aper_exs, 
                         Illustris_dir):
    """Get all the necessary 1-D profiles for single tng galaxy."""
    print("# Dealing with galaxy %d" % idx)
    
    # Get the FITS files
    fits_cen = get_fits_name(idx, img_type='cen', Illustris_dir=Illustris_dir)
    fits_ins = get_fits_name(idx, img_type='ins', Illustris_dir=Illustris_dir)
    fits_exs = get_fits_name(idx, img_type='exs', Illustris_dir=Illustris_dir)

    # Step 2, Step 3 for central galaxy
    ell_cen = Illustris_get_1d_prof(fits_cen, aper_cen)
    ell_cen_3, bin_cen_3 = ell_cen

    # Step 2, Step 3 for In-situ component
    ell_ins = Illustris_get_1d_prof(fits_ins, aper_ins)
    ell_ins_3,bin_ins_3 = ell_ins

    # Step 2, Step 3 for Ex-situ component
    ell_exs = Illustris_get_1d_prof(fits_exs, aper_exs)
    ell_exs_3,  bin_exs_3 = ell_exs

    # Step 4 force 1-D profiles for in-situ and ex-situ components
    ell_ins_4, bin_ins_4 = Illustris_get_1d_force(fits_ins, bin_cen_3, aper_cen)
    ell_exs_4, bin_exs_4 = Illustris_get_1d_force(fits_exs, bin_cen_3, aper_cen)
    
    return {'ell_cen_3': ell_cen_3,
            'ell_ins_3': ell_ins_3,
            'ell_exs_3': ell_exs_3,
            'ell_ins_4': ell_ins_4, 'ell_exs_4': ell_exs_4,
            'bin_cen_3': bin_cen_3}

### Getting the profiles for low resolution map

In [ ]:
Illustris_0_ell_quick = [Illustris_get_1d_all(idx,
                            Illustris_0_cen_aper[idx], 
                            Illustris_0_ins_aper[idx], 
                            Illustris_0_exs_aper[idx], 
                            Illustris_dir=illustris_quick_direc)for idx in np.arange(n_galaxies)] 

### Getting the profiles for high resolution map

In [ ]:
Illustris_0_ell_high = [Illustris_get_1d_all(idx,
                            Illustris_0_cen_aper_part[idx], 
                            Illustris_0_ins_aper_part[idx], 
                            Illustris_0_exs_aper_part[idx], 
                            Illustris_dir=illustris_high_direc)for idx in np.arange(n_galaxies)] 

In [ ]:
np.save(os.path.join(illustris_quick_direc, 'Illustris_0_ell.npy'), Illustris_0_ell_quick)

In [ ]:
np.save(os.path.join(illustris_high_direc, 'Illustris_0_ell.npy'), Illustris_0_ell_high)